## Q1: What is the salary distribution of one department?
1. createSalaryCompareHist with one department

## Q2: What is the salary distribution across several department?
1. createSalaryCompareHist with more than one department

## Q3: What is the size of the department across several department?
1. createFigDiffDepAndCampus

## Q4: List the median salary of all department/all job in a campus
1. createFigListAllDepSalary

In [1]:
import numpy as np
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

from ipywidgets import widgets 

from IPython.display import display, clear_output, Image
from plotly.widgets import GraphWidget

plotly.tools.set_credentials_file(username='wangqiwen', api_key='8r40jKGvIemGyOP1PEZ1')

<IPython.core.display.Javascript object>

In [2]:
# import csv data
SalaryOverYears = pd.read_csv("https://raw.githubusercontent.com/qwang70/hackculture/master/csv/SalaryOverYears.csv")
print(SalaryOverYears)

                             College            Campus  \
0      Agr, Consumer, & Env Sciences  Urbana-Champaign   
1      Agr, Consumer, & Env Sciences  Urbana-Champaign   
2      Agr, Consumer, & Env Sciences  Urbana-Champaign   
3      Agr, Consumer, & Env Sciences  Urbana-Champaign   
4      Agr, Consumer, & Env Sciences  Urbana-Champaign   
5      Agr, Consumer, & Env Sciences  Urbana-Champaign   
6      Agr, Consumer, & Env Sciences  Urbana-Champaign   
7      Agr, Consumer, & Env Sciences  Urbana-Champaign   
8      Agr, Consumer, & Env Sciences  Urbana-Champaign   
9      Agr, Consumer, & Env Sciences  Urbana-Champaign   
10     Agr, Consumer, & Env Sciences  Urbana-Champaign   
11     Agr, Consumer, & Env Sciences  Urbana-Champaign   
12     Agr, Consumer, & Env Sciences  Urbana-Champaign   
13     Agr, Consumer, & Env Sciences  Urbana-Champaign   
14     Agr, Consumer, & Env Sciences  Urbana-Champaign   
15     Agr, Consumer, & Env Sciences  Urbana-Champaign   
16     Agr, Co

In [5]:
def getSalary(department, campus, year_col):
    salary = SalaryOverYears.loc[\
                (SalaryOverYears['Department'] == department) \
                & (SalaryOverYears['Campus'] == campus), year_col].dropna()
    return salary


def createFigHist(departments, campus, year = 2017):
    assert len(departments) == len(campus)
    
    # if year is a int, cast to a list
    if type(year) is int:
        year = [year]*len(campus)
    else:
        assert type(year) is list
        assert len(year) == len(campus)
        
    # get the salary
    data = []
    for idx in range(len(departments)):
        year_col = "Salary" + str(year[idx])
        salary = getSalary(departments[idx], campus[idx], year_col)
        data.append(Histogram(
            x=salary,
            histnorm='percent',
            name=departments[idx],
            opacity=0.75
        ))

    layout = Layout(
        title='Salary Range of Faculty in {}'.format(' & '.join(departments)),
        xaxis=dict(
            title='Salary'
        ),
        yaxis=dict(
            title='Percentage of Faculty'
        ),
        bargap=0.2,
        bargroupgap=0.1
    )
    fig = Figure(data=data, layout=layout)
    return fig

def createFigDiffDepAndCampus(departments, campus, year=2017):
    assert len(departments) == len(campus)
    
    # if year is a int, cast to a list
    if type(year) is int:
        year = [year]*len(campus)
    else:
        assert type(year) is list
        assert len(year) == len(campus)
        
    # get the salary
    populationSize = []
    labels = []
    medians = []
    for idx in range(len(departments)):
        year_col = "Salary" + str(year[idx])
        salary = getSalary(departments[idx], campus[idx], year_col)
        populationSize.append(len(salary))
        median = salary.median()
        medians.append(int(median))
        labels.append('{} - {}'.format(campus[idx], departments[idx]))
    # parameters used to create graph
        
    population_bar = Bar(
            x = labels,
            y = populationSize,  
            name="population size",       
            opacity=0.75,    
           )
    salary_scatter = Scatter(
            x = labels,
            y = medians,
            name="median salary",
            # marker + lines
            mode = 'lines+markers',
            line = dict(
                width = 4,),
            yaxis='y2',           
            opacity=0.75
)
    layout = Layout(
        title= "Number of Faculty and Median Salary in the Dataset",
        xaxis=dict(
            title='Department',
        ),
        yaxis=dict(
            title='Number of Faculty'
        ),
        yaxis2=dict(
            title='Median Salary',
            overlaying='y',
            side='right',
            
            # label color
            titlefont=dict(
                color='#ff7f0e'
            ),
            tickfont=dict(
                color='#ff7f0e'
            ),
        ),
        margin=Margin(
         b=160,
        ),
    )
    fig = Figure(data=[population_bar, salary_scatter], layout=layout)
    return fig
    
def createAllDepSalaryTrace(campus = "Urbana-Champaign", year = 2017, top = None):
    year_col = "Salary" + str(year)
    
    
    # get the median salary of each department
    gb = SalaryOverYears.loc[SalaryOverYears['Campus'] == campus, ["Department", year_col]].dropna()\
                .groupby(['Department'])
    agg_df = gb.agg(['median', 'count'])
    agg_df = agg_df[agg_df[(year_col, 'count')] > 5]
    sorted_depMedian = agg_df.sort_values((year_col, 'median'), ascending = False)
    x = sorted_depMedian[(year_col, 'median')].tolist()
    y = sorted_depMedian.index.tolist()
        
    # select top subset elements
    if top:
        x = x[:top]
        y = y[:top]

    x.reverse()
    y.reverse()
    return Bar(
        y=y,
        x=x,
        name='Median Salary',
        orientation = 'h',
        opacity=0.75
    )

def createFigListAllDepSalary(campus = "Urbana-Champaign", year = 2017, top = None):
    assert year >= 2013 and year <= 2017
    
    trace1 = createAllDepSalaryTrace(top=top)
    trace2 = createAllDepSalaryTrace("Chicago", year, top=top)
    trace3 = createAllDepSalaryTrace("Springfield", year, top=top)

    fig = tls.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.05,\
              subplot_titles=('{} Median Salary of Faculty Members in Urbana-Champaign'.format(year), \
                              '{} Median Salary of Faculty Members in Chicago'.format(year), \
                              '{} Median Salary of Faculty Members in Springfield'.format(year)))

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(trace3, 3, 1)

    fig['layout'].update( \
        height=1200, width=600,
        margin=Margin(\
            l=180, \
            pad=5,
            ),\
        showlegend=False,)
    return fig
        
def createAllDepJobSalaryTrace(campus = "Urbana-Champaign", year = 2017, job = "ASST PROF", top = None):
    year_col = "Salary" + str(year)
    
    # get the median salary of each job & departmentdepartment
    gb = SalaryOverYears.loc[\
             (SalaryOverYears['Campus'] == campus) & (SalaryOverYears['JobTitle'] == job), \
             ["Department",year_col]].dropna()\
            .groupby(['Department'])
    agg_df = gb.agg(['median', 'count'])
    agg_df = agg_df[agg_df[(year_col, 'count')] > 2]
    sorted_depMedian = agg_df.sort_values((year_col, 'median'), ascending = False)
    x = sorted_depMedian[(year_col, 'median')].tolist()
    y = sorted_depMedian.index.tolist()

    # select top subset elements
    if top:
        x = x[:top]
        y = y[:top]

    x.reverse()
    y.reverse()
    return Bar(
        y=y,
        x=x,
        name='Median Salary',
        orientation = 'h',

        marker=dict(
            color='rgba(50, 171, 96, 0.7)',
            line=dict(
                color='rgba(50, 171, 96, 1.0)',
            )
        ),
        opacity=0.75
    )

def createFigListAllDepJobSalary(year = 2017, job = "ASST PROF", top = None):
    trace1 = createAllDepJobSalaryTrace(top=top)
    trace2 = createAllDepJobSalaryTrace("Chicago", year, job, top=top)
    trace3 = createAllDepJobSalaryTrace("Springfield", year, job, top=top)

    fig = tls.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.05, \
                          subplot_titles=('{} Median Salary of {} in Urbana-Champaign'.format(year, job), \
                                          '{} Median Salary of {} in Chicago'.format(year, job), \
                                          '{} Median Salary of {} in Springfield'.format(year, job)))

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(trace3, 3, 1)

    fig['layout'].update( \
        height=1200, width=600,
        margin=Margin(\
            l=180, \
            pad=5,
            ),\
        showlegend=False,
        )
    return fig


In [6]:
# define widgets
def createIntSliderForYear():
    return widgets.IntSlider(
        value=2017,
        min=2013,
        max=2017,
        step=1,
        description='Year:',
        continuous_update = False
    )

def createTextBoxForDep(v="Mathematics"):
    return widgets.Text(
        description='Department:   ',
        value=v,
    )

def createDropDownForCampus():
    return widgets.Dropdown(
        options=["Chicago", "Springfield", "Urbana-Champaign"],
        value='Urbana-Champaign',
        description='Campus:',
    )

def createBundle(v="Mathematics"):
    return [ createDropDownForCampus(), createTextBoxForDep(v), createIntSliderForYear()]


# for job select widget
def createDropDownForJobs():
    jobs = ['PROF', 'ASSOC PROF', 'ASST PROF', 'LECTURER', 'ASSOC DIR',
       'CLIN ASST PROF', 'SR LECTURER', 'RES SCI', 'RES ASST PROF',
       'SR RES SCI', 'POSTDOC RES ASSOC', 'CLIN PROF',
       'ASSOC PROF UNIV LIBRARY', 'ASST PROF UNIV LIBRARY', 'DEAN',
       'JL DOOB RES ASST PROF', 'CLIN ASSOC PROF', 'VST ASST PROF',
       'RES ASSOC PROF', 'ASST DEAN', 'RES PROF', 'PROF (RT)']
    return widgets.Dropdown(
        options=jobs,
        value='ASST PROF',
        description='Job Title:',
    )


def createIntTop():
    return widgets.IntText(
        value=10,
        description='Top X:',
        disabled=False
    )

In [5]:
# fig = createFigHist(['Information Sciences'], year = 2017)
# salaryHist = py.iplot(fig, filename='salaryHist')
# salaryHist_url = salaryHist.resource

# import fig
salaryHist_url = "https://plot.ly/~wangqiwen/22/salary-range-of-faculty-in-information-sciences/"
# Assign an emptry graph widget with two traces
g_salary_dist = GraphWidget(salaryHist_url)

# the widget for the distribution of the salary
salary_dist_year_container = createIntSliderForYear()

salary_dist_department_one_textbox = createTextBoxForDep()

salary_dist_campus_dropdown = createDropDownForCampus()


def response_salary_dist(change):
    year = salary_dist_year_container.value
    department = [salary_dist_department_one_textbox.value]
    campus = [salary_dist_campus_dropdown.value]
    fig = createFigHist(department, campus=campus, year=year)
    if len(fig.data[0].x) > 0:
        g_salary_dist.restyle({'x': [list(fig.data[0].x)], "name": fig.data[0].name})
        g_salary_dist.relayout(fig.layout)

# invoke changes
salary_dist_year_container.observe(response_salary_dist, names="value")
salary_dist_department_one_textbox.observe(response_salary_dist, names="value")
salary_dist_campus_dropdown.observe(response_salary_dist, names="value")

# define layout container
salary_dist_container = widgets.HBox([salary_dist_campus_dropdown, salary_dist_department_one_textbox, salary_dist_year_container])
display(salary_dist_container)
display(g_salary_dist)

GraphWidget()

In [6]:
# call the create fig function
# fig = createFigHist(['Information Sciences', 'Computer Science'])
# py.iplot(fig, filename='salaryHist3')

# import graph
salary_compare_url = "https://plot.ly/~wangqiwen/8/salary-range-of-faculty-in-mathematics-information-sciences/"
g_salary_compare = GraphWidget(salary_compare_url)

# create widget
tab_contents = ['Department 1', 'Department 2']

bundleList = [createBundle("Mathematics"), createBundle("Information Sciences")]
children = [widgets.HBox(b) for b in bundleList]
tab = widgets.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])

def response_salary_compare(change):
    # initialize
    campus = []
    departments = []
    years = []

    for bundle in bundleList:
        if(len(bundle[1].value)):
            campus.append(bundle[0].value)
            departments.append(bundle[1].value)
            years.append(bundle[2].value)
    fig = createFigHist(departments, campus=campus, year=years)
    if len(fig.data) == 2 and len(fig.data[0].x) > 0 and len(fig.data[1].x) > 0:
        for idx in range(len(fig.data)):
            g_salary_compare.restyle({'x': [list(fig.data[idx].x)], "name": fig.data[idx].name}, indices = idx)
        g_salary_compare.relayout(fig.layout)

# invoke changes
for bundle in bundleList:
    for ele in bundle:
        ele.observe(response_salary_compare, names="value")

# display
display(tab)

display(g_salary_compare)

GraphWidget()

In [7]:
# Q3: What is the size of Computer Science Science department across UIC, UIS, UIUC?
# fig = createFigDiffDepAndCampus(['History']*3, \
#                          ["Chicago", "Springfield", "Urbana-Champaign"], year=2017)
# py.iplot(fig, filename='salaryCampus')

# import graph
salary_population_history = "https://plot.ly/~wangqiwen/14"
g_salary_population_history = GraphWidget(salary_population_history)
display(g_salary_population_history)

GraphWidget()

In [4]:
# Q3: What is the size of Information Science department across UIC, UIS, UIUC?
#fig = createFigDiffDepAndCampus(\
#        ["Management Information Systems", "Information/Decision Sciences", "Information Sciences"], \
#        ["Springfield", "Chicago", "Urbana-Champaign"], year=2017)
#py.iplot(fig, filename='salaryCampus')

# import graph
salary_population_is = "https://plot.ly/~wangqiwen/24"
g_salary_population_is = GraphWidget(salary_population_is)
display(g_salary_population_is)

In [7]:
# fig = createFigListAllDepSalary(top = 10)
# py.iplot(fig, filename='listSalaryBar')

# import graph
list_dep_salary = "https://plot.ly/~wangqiwen/18/median-salary-median-salary-median-salary/"
g_list_dep_salary = GraphWidget(list_dep_salary)
display(g_list_dep_salary)

GraphWidget()

In [8]:
#fig = createFigListAllDepJobSalary(top = 10)
#py.iplot(fig, filename='listSalaryBarJob')


# import graph
list_dep_job_salary = "https://plot.ly/~wangqiwen/20/median-salary-median-salary-median-salary/"
g_list_dep_job_salary = GraphWidget(list_dep_job_salary)

def response_job(change):
    year = salary_job_year_slider.value
    job = salary_job_dropdown.value
    top = salary_job_top.value
    
    # traces
    trace1 = createAllDepJobSalaryTrace(year=year, job=job, top=top)
    trace2 = createAllDepJobSalaryTrace("Chicago", year=year, job=job, top=top)
    trace3 = createAllDepJobSalaryTrace("Springfield", year=year, job=job, top=top)
    
    # update subplot titles
    subplot_titles=('{} Median Salary of {} in Urbana-Champaign'.format(year, job), \
                  '{} Median Salary of {} in Chicago'.format(year, job), \
                  '{} Median Salary of {} in Springfield'.format(year, job))
    
    # update style and layout
    g_list_dep_job_salary.restyle({'x': [list(trace1.x)], 'y': [list(trace1.y)]}, indices = [0])
    g_list_dep_job_salary.restyle({'x': [list(trace2.x)], 'y': [list(trace2.y)]}, indices = [1])
    g_list_dep_job_salary.restyle({'x': [list(trace3.x)], 'y': [list(trace3.y)]}, indices = [2])
    g_list_dep_job_salary.relayout({'annotations[0].text': subplot_titles[0], \
                                    'annotations[1].text': subplot_titles[1], \
                                    'annotations[2].text': subplot_titles[2],})

# define layout container
salary_job_dropdown = createDropDownForJobs()
salary_job_year_slider = createIntSliderForYear()
salary_job_top = createIntTop()
salary_job_container = widgets.HBox([salary_job_dropdown, salary_job_year_slider, salary_job_top])

# invoke changes
salary_job_dropdown.observe(response_job, names="value")
salary_job_year_slider.observe(response_job, names="value")
salary_job_top.observe(response_job, names="value")


display(salary_job_container)
display(g_list_dep_job_salary)


GraphWidget()